In [1]:
import os

os.environ["USE_PYGEOS"] = "0"
os.environ["LD_LIBRARY_PATH"] = "/usr/lib/x86_64-linux-gnu/libstdc++.so.6"

In [2]:
import pandas as pd
import shapely
import networkx as nx
import matplotlib.pyplot as plt
import json
from utils.functions import *
from pyrosm import OSM, get_data
from decouple import config
import os
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import seaborn as sns
import osmnx as ox

### Set variables 

In [4]:
# Place
place = "Oslo"


# If specific area specified
sub_areas = [
    "Alna",
    "Bjerke",
    #     "Frogner",
    #     "Gamle Oslo",
    #     "Grorud",
    #     "Grünerløkka",
    #     "Nordre Aker",
    #     "Nordstrand",
    #     "Sagene",
    #     "St. Hanshaugen",
    #     "Stovner",
    #     "Søndre Nordstrand",
    #     "Ullern",
    #     "Vestre Aker",
    #     "Østensjø",
]

In [5]:
# Load self-defined lane types and criterias
with open("categories.json", "r") as f:
    categories = json.load(f)

In [6]:
# Attributes
attributes = list(
    set(
        [
            colname
            for category in categories
            for colname, value in category["criteria"].items()
        ]
    )
)

In [7]:
# Define the roads and their order for bicyclists
# TODO: Check if this can be added as environment variable or specified by user?
category_score = {
    "Designated cycleway, segregated": 1,
    "Bicycle and footpath": 2,
    "Road with bike lane": 3,
    "Road with one-directional cycleway": 4,
    "Footpath": 5,
    "Sidewalk": 6,
    "Pedestrian way": 7,
    "Road without bike lane": 8,
}

parent_category_score = {
    "Designated cyclepath": 1,
    "Footway": 2,
    "Road without bike lane": 3,
}


# Add road score to categories
for category, score in category_score.items():
    for road in categories:
        if road["category"] == category:
            road["categoryScore"] = score

# Add road score to parent categories
for parent, score in parent_category_score.items():
    for road in categories:
        if road["parent"] == parent:
            road["parentScore"] = score

### OSM Data Retrieval

In [8]:
# Initialize the OSM object
fp = os.path.join("data", "OpenStreetMap", f"{place}.osm.pbf")
osm = OSM(fp)

In [9]:
# Set bounding boxes from specified sub-areas
bounding_boxes = []
if sub_areas:
    for a in sub_areas:
        bounding_box = osm.get_boundaries(name=a)
        bounding_boxes.append(bounding_box["geometry"].values[0])

    # Merge bounding boxes into one
    bb = shapely.ops.unary_union(bounding_boxes)
else:
    bb = None

In [10]:
# Initiliaze with bounding box
osm = OSM(fp, bounding_box=bb)

In [11]:
bike_nodes, bike_edges = osm.get_network(
    network_type="all", nodes=True, extra_attributes=attributes
)

In [12]:
roads_to_keep = [
    "cycleway",
    "footway",
    "secondary",
    "tertiary",
    "unclassified",
    "sidewalk",
    "pedestrian",
    "service",
    "residential",
]

In [13]:
# Filter out un-bikeable roads
bike_edges = bike_edges[bike_edges["highway"].isin(roads_to_keep)]
bike_edges = bike_edges[bike_edges["access"].isnull()]

print(f"Number of edges before nodes filtering: {len(bike_edges)}")
print(f"Number of nodes before nodes filtering: {len(bike_nodes)}\n")


# Filter out nodes that are not in edges
nodes_in_edges = [u for u in bike_edges["u"]] + [v for v in bike_edges["v"]]
bike_nodes = bike_nodes[bike_nodes["id"].isin(nodes_in_edges)]

print(f"Number of edges after edge filtering: {len(bike_edges)}")
print(f"Number of nodes after edge filtering: {len(bike_nodes)}\n")

# Filter out edges not are not in nodes
nodes_in_nodes = [u for u in bike_nodes["id"]]
bike_edges = bike_edges[
    bike_edges["u"].isin(nodes_in_nodes) & bike_edges["v"].isin(nodes_in_nodes)
]

print(f"Number of edges after node filtering: {len(bike_edges)}")
print(f"Number of nodes after node filtering: {len(bike_nodes)}\n")

Number of edges before nodes filtering: 53329
Number of nodes before nodes filtering: 57381

Number of edges after edge filtering: 53329
Number of nodes after edge filtering: 49888

Number of edges after node filtering: 53071
Number of nodes after node filtering: 49888



In [14]:
bike_edges = assign_categories(bike_edges, categories, "category")
bike_edges = assign_categories(bike_edges, categories, "parent")

bike_edges = set_category_score(bike_edges, categories, "category")
bike_edges = set_category_score(bike_edges, categories, "parent")

In [20]:
bike_edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 53071 entries, 39 to 61960
Data columns (total 44 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   access          0 non-null      object  
 1   area            468 non-null    object  
 2   bicycle         5402 non-null   object  
 3   bridge          486 non-null    object  
 4   cycleway        197 non-null    object  
 5   foot            6715 non-null   object  
 6   footway         9009 non-null   object  
 7   highway         53071 non-null  object  
 8   junction        762 non-null    object  
 9   lanes           2630 non-null   object  
 10  lit             8659 non-null   object  
 11  maxspeed        5823 non-null   object  
 12  motorcar        0 non-null      object  
 13  motorroad       0 non-null      object  
 14  motor_vehicle   1051 non-null   object  
 15  name            13736 non-null  object  
 16  oneway          3346 non-null   object  
 17  overtaki

In [36]:
# Trim GDF
edge_cols_to_keep = [
    "id",
    "u",
    "v",
    "highway",
    "category",
    "categoryScore",
    "parent",
    "parentScore",
    "lit",
    "oneway",
    "oneway:bicycle",
    "segregated",
    "surface",
    "length",
    "geometry",
]

node_cols_to_keep = [
    "timestamp",
    "changeset",
    "version",
    "visible",
    "lon",
    "lat",
    "id",
    "geometry",
]

bike_edges = bike_edges[edge_cols_to_keep]
node_edges = node_edges[node_cols_to_keep]

In [37]:
# Save GeoDataFrames as GeoJSON
with open(f"data/processed/{place}_nodes.geojson", "w") as f:
    f.write(bike_nodes.to_json(indent=4))

with open(f"data/processed/{place}_edges.geojson", "w") as f:
    f.write(bike_edges.to_json(indent=4))

In [38]:
n_edges = len(bike_edges)
n_nodes = len(bike_nodes)

print(f"Number of edges: {n_edges}")
print(f"Number of nodes: {n_nodes}")

Number of edges: 352208
Number of nodes: 326813


In [39]:
# Plot bike road category length
plt.figure(figsize=(10, 10))
sns.barplot(
    x=bike_edges["length"],
    y=bike_edges["category"],
    estimator=sum,
    errorbar=None,
    orient="h",
)
# Save the figure
figpath = os.path.join("figures", f"{place}_road_lengths.png")
plt.savefig(figpath, dpi=200, bbox_inches="tight")
plt.close()

### Visualisations

In [40]:
plot_bike_roads_with_category(
    bike_edges,
    place,
    "parent",
    "parentScore",
    cmap_name="Oranges_r",
    bg_color="black",
)

In [41]:
m = bike_edges[["id", "parent", "geometry"]].explore(column="parent")

outfp = f"figures/{place}_bike_roads_interactive.html"
m.save(outfp)

# webbrowser.open(outfp)

### Convert to graph and process data

In [42]:
G = osm.to_graph(bike_nodes, bike_edges, graph_type="networkx")

In [43]:
# Test plot route
orig = ox.distance.nearest_nodes(G, 10.82746, 59.94903)
# orig = ox.distance.nearest_nodes(G, 10.79564, 59.92986)
dest = ox.distance.nearest_nodes(G, 10.82817, 59.92831)
# dest = ox.distance.nearest_nodes(G, 10.8057, 59.931)

start_time = time.time()
route = ox.distance.shortest_path(G, orig, dest, weight="length", cpus=1)
end_time = time.time()
run_time = end_time - start_time
print(run_time)

if route != None:
    plt.figure()
    fig, ax = ox.plot_graph_route(
        G,
        route,
        route_color="r",
        route_linewidth=1,
        node_size=0,
        edge_linewidth=0.1,
        orig_dest_size=5,
        figsize=(30, 30),
        show=False,
        save=True,
        filepath=f"figures/{place}_route_osmnx.png",
    )
    plt.close()

elif route == None:
    print("No route found. Could not plot")

1.0418646335601807
